In [1]:
import Toy_System_Fully_Continuous as ts
import numpy as np
from multi_step import *
from utils import objective, stage

Using backend: tensorflow


In [2]:
# Generate the toy problem class object

toy = ts.Toy()

In [3]:
# Generate conditions to be used for generating data

# Define the condition ranges [lower,upper,step (if applicable)]
#[T1, P1, t_out1, T2, P2, t_out2, T3, P3, t_out3]
condition_ranges = [[2,10],[2,10],[2,10],[2,10],[2,10],[2,10],[2,10],[2,10],[2,10]]

# Generate all condition permutations
# conditions = toy.allConditions(condition_ranges)

# Generate values to explore the search space using low discrepancy
# quasi-random numbers and discrete methods
num_points = 1000
conditions =  toy.sobolSequenceConditions(condition_ranges, num_points)

In [4]:
# Generate the yields of the various materials
yieldList = []
costList = []
for conditions_run in conditions:
    yields, costs = toy.experimentalRun(1,conditions_run)
    yieldList.append(yields)
    costList.append(costs)
yieldList = np.array(yieldList)

In [5]:
# Fit the first stage data
x1 = conditions[:,:3]
y1 = yieldList[:,2:5]
toy.fit_stage_1([x1,y1], epochs=5)

epoch:  0 , loss =  6.9983377


In [6]:
# Fit the second stage data

x2 = np.concatenate((np.expand_dims(yieldList[:,3],axis=1),conditions[:,3:6]),axis=1)
y2 = yieldList[:,5:9]
toy.fit_stage_2([x2,y2], epochs=5)

epoch:  0 , loss =  8.475381


In [7]:
# Fit the third stage data
x3 = np.concatenate((np.expand_dims(yieldList[:,4],axis=1),yieldList[:,7:9],conditions[:,6:]),axis=1)
y3 = yieldList[:,9:]
toy.fit_stage_3([x3,y3], epochs=5)

epoch:  0 , loss =  5.163458


In [8]:
#Plot stage 1
# num_plot_points = 20
# plot_var = 0
# plot_x = np.hstack((np.expand_dims(np.linspace(2,10,num_plot_points),1),np.full((num_plot_points,1),7),np.full((num_plot_points,1),5)))
# plot_y = np.array([toy.stage_1(0.4, *row) for row in plot_x])
# toy.stage_1_model.visualise(plot_x,plot_y,plot_x,plot_var,'Stage_1')

In [9]:
# Save the models
# toy.stage_1_model.save('../examples/Toy_problem/Saved_Models/stage_1_model')
# toy.stage_2_model.save('../examples/Toy_problem/Saved_Models/stage_2_model')
# toy.stage_3_model.save('../examples/Toy_problem/Saved_Models/stage_3_model')

In [10]:
# Generate the multi-step class
multi_step = multi_step('adam')

In [11]:
# Load the models from a save
# import tensorflow as tf
# toy = ts.Toy()
# 
# toy.stage_1_model = tf.keras.models.load_model('../examples/Toy_problem/Saved_Models/stage_1_model.h5')
# toy.stage_2_model = tf.keras.models.load_model('../examples/Toy_problem/Saved_Models/stage_2_model.h5')
# toy.stage_3_model = tf.keras.models.load_model('../examples/Toy_problem/Saved_Models/stage_3_model.h5')

In [12]:
# Load the models into the framework

#Define the stages
# stageID, stageModel, conditionRanges, followingStages (followingStageID, feedingVariables)
stage1 = stage(0, toy.stage_1_model, condition_ranges[0:3], [[1, [1]], [2, [2]]])
stage2 = stage(1, toy.stage_2_model, condition_ranges[3:6], [[2,[2,3]]])
stage3 = stage(2, toy.stage_3_model, condition_ranges[6:])

stages = [stage1, stage2, stage3]

multi_step.loadModels(stages)

C:\Users\geeso\Anaconda3\envs\multi-step\lib\site-packages\dgl\base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


In [13]:
# Define the objective functions
def objectiveFunction1(var1, var2, var3):
    return (var1+var2)**2+(var2+var3)**2
    
def objectiveFunction2(var1, var2, var3, var4):
    return (var1+var4)**2-var3**2+var2**2

# Define the objective objects including the objective functions and the IDs of relevant variables/outputs
# Define objective variables with form: [stage, 'input'/'output', stage variable]
objective1 = objective(objectiveFunction1, [[stage1,'inputs',1],[stage2,'inputs',1],[stage3,'inputs',1]])
objective2 = objective(objectiveFunction2,[[stage1,'outputs',1],[stage2,'outputs',0],[stage3,'inputs',1],[stage2,'outputs',1]])

objectives = [objective1, objective2]

In [14]:
# Define the objectives for the framework
multi_step.defineObjectives(objectives)

In [ ]:
# Optimise the process
# numObjectives, numObjectiveSamples, preferenceCodeDim, hyperparameterGeneratorDim, epochs=5, h_dim=5, n_layers=4, network_type='GATConv', **kwargs
multi_step.train(2, 10, 10, 100, epochs=50, h_dim=5, n_layers=4, network_type='GATConv', num_heads=3)

Weightings: [0.6128444  0.38715556], Epoch: 0, Loss: -163.74339294433594
Weightings: [0.6128444  0.38715556], Epoch: 10, Loss: -452.4247741699219
Weightings: [0.6128444  0.38715556], Epoch: 20, Loss: -138.3848114013672
Weightings: [0.6128444  0.38715556], Epoch: 30, Loss: -138.15211486816406
Weightings: [0.6128444  0.38715556], Epoch: 40, Loss: -138.21026611328125
Weightings: [0.0339484 0.9660516], Epoch: 0, Loss: 85.81096649169922
Weightings: [0.0339484 0.9660516], Epoch: 10, Loss: 90.06233215332031
Weightings: [0.0339484 0.9660516], Epoch: 20, Loss: 67.1224594116211
Weightings: [0.0339484 0.9660516], Epoch: 30, Loss: -16.67315101623535
Weightings: [0.0339484 0.9660516], Epoch: 40, Loss: -16.742063522338867
Weightings: [0.6692051  0.33079487], Epoch: 0, Loss: -363.4464416503906
Weightings: [0.6692051  0.33079487], Epoch: 10, Loss: -363.4673767089844
Weightings: [0.6692051  0.33079487], Epoch: 20, Loss: -363.4741516113281
Weightings: [0.6692051  0.33079487], Epoch: 30, Loss: -363.42010